In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel
from scipy.stats import friedmanchisquare
from scipy.stats import wilcoxon
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from matplotlib.patches import Ellipse

# Load your file
pdf = pd.read_csv("data/NPXpp_with_timecode.csv")
# labels = {0:'Baseline', 1:'Post 1/3', 2:'D30'}
pdf.columns = pdf.columns.str.strip()
bdf = pd.read_excel("data/blood_all_timepoints.xlsx")
bdf.columns = bdf.columns.str.strip()
bdf = bdf.applymap(lambda x: x.strip() if isinstance(x, str) else x)
sedf = pd.read_excel("data/Seahorse_results.xlsx")
sedf.columns = sedf.columns.str.strip()

print(" File read done")
#print(bdf.columns.tolist())

 File read done


C:\Users\mondalsy\AppData\Local\Temp\ipykernel_9908\1561341393.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bdf = bdf.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [33]:
#sedf

In [111]:

sigpdf = pd.read_csv("data/anova_results_sig_proteins.csv")
# Extract the first column as a list
sig_protein = sigpdf.iloc[:, 0].tolist()
#print(sig_protein)
sig_blood = ['NEUT%(%)', 'EO%(%)', 'NEUT#(10^9/L)', 'LYMPH%(%)', 'WBC(10^9/L)', 'EO#(10^9/L)', 'BASO%(%)', 'MONO#(10^9/L)',
                  'LYMPH#(10^9/L)', 'IG#(10^9/L)', 'HCT(L/L)', 'RBC(10^12/L)', 'HGB(g/L)']
#sig_blood
# this include 3 Annova+ masslin2 features union
sig_seahorse = [ 'Basal glycolysis', 'Maximal glycolitic capacity', 'ATP restricted glycolisis', 
                 'ATP production rate', 'glycoATP production rate (=glycoPER)', 'PER', 'mitoPER', 
                 'Basal mitochondrial respiration', 'Phosphorylation-linked respiration', 
                 'Maximal respiration','ATP-linked respiration', 'mitoOCR', 'OCR ATP']

print(" features read")
#sig_seahorse
len(sig_protein)

 features read


229

In [ ]:
 pdf, bdf (1st 2 column same), sedf (Patient ID,Timepoint,	Treatment) form input with features amoung need to select sig_protein features from pdf
sig_blood from bdf and sig_seahorse from sedf. need to read and assign to new pdf to v, bdf to bd_sig_features, sedf to metab_sig_features

In [115]:
# Filter the datasets ---
# For pdf: select only sig_protein columns (first 2 columns are kept)
prot_sig_features = pdf.loc[:, pdf.columns[:2].tolist() + [col for col in pdf.columns if col in sig_protein]]
# For bdf: select only sig_blood columns (first 2 columns are assumed same)
bd_sig_features = bdf.loc[:, bdf.columns[:2].tolist() + [col for col in bdf.columns if col in sig_blood]]
# For sedf: select only sig_seahorse columns (keep Patient ID, Timepoint, Treatment)
metab_sig_features = sedf.loc[:, sedf.columns[:3].tolist() + [col for col in sedf.columns if col in sig_seahorse]]
# --- Step 4: Check the results ---
print(" significant features only:")
#print(prot_sig_features.head())
#print(bd_sig_features.head())
#print(metab_sig_features.head())
metab_sig_features.shape[1]


 significant features only:


16

In [116]:
# for seahorse df only ctrl vs stim column concat with sig features only
#in df metab_sig_features i have sig_seahorse features where Treatment column 
#i have control, stim value. so i want to concat sig_seahorse features with like ctrl_* stim_* *is 
#the features. i want to match ['PatientID', 'Timepoint'] basis so can ignore Treatment column like this.


# need to run prev block
metab_sig_features = (
    metab_sig_features
    # reshape to long so Treatment can be used for naming
    .set_index(['PatientID', 'Timepoint', 'Treatment'])
    .stack()
    .reset_index(name='Value')
)

# Create ctrl_* and stim_* column names
metab_sig_features['Feature_Treat'] = (
    metab_sig_features['Treatment'].str.lower().str[:4] + '_' +
    metab_sig_features['level_3']
)

# Pivot to sample-wise wide format (Treatment is dropped here)
metab_sig_features = (
    metab_sig_features
    .pivot_table(
        index=['PatientID', 'Timepoint'],
        columns='Feature_Treat',
        values='Value',
        aggfunc='first'
    )
    .reset_index()
)

# Clean column index
metab_sig_features.columns.name = None
metab_sig_features.to_csv("run1/metab_sig_features.csv", index=True)
metab_sig_features.shape[1]


28

In [46]:
#bd_sig_features

# need to do  normalization → Z-score for blood Proteomics Metabolomics

In [105]:
# Blood — Separate numeric & non-numeric
non_numeric_cols = ['PatientID', 'Timepoint']  # adjust names if needed
bnumeric_cols = [col for col in bd_sig_features.columns if col not in non_numeric_cols]
pnumeric_cols = [col for col in prot_sig_features.columns if col not in non_numeric_cols]

senon_numeric_cols = ['PatientID', 'Timepoint', "Treatment"]  # adjust names if needed
senumeric_cols = [col for col in metab_sig_features.columns if col not in senon_numeric_cols]

# 3. Convert numeric columns safely
for col in bnumeric_cols:
    bd_sig_features[col] = pd.to_numeric(bd_sig_features[col], errors='coerce')

for col in pnumeric_cols:
    prot_sig_features[col] = pd.to_numeric(prot_sig_features[col], errors='coerce')

# for seahorse
for col in senumeric_cols:
    metab_sig_features[col] = pd.to_numeric(metab_sig_features[col], errors='coerce')

# StandardScaler
scaler = StandardScaler()
bd_scaled = scaler.fit_transform(bd_sig_features[bnumeric_cols])
prot_scaled = scaler.fit_transform(prot_sig_features[pnumeric_cols])
metab_scaled = scaler.fit_transform(metab_sig_features[senumeric_cols])

# --- 1. Recreate scaled dataframes with metadata ---
bd_scaled_df = pd.concat([
    bd_sig_features[['PatientID', 'Timepoint']].reset_index(drop=True),
    pd.DataFrame(bd_scaled, columns=bnumeric_cols)
], axis=1)

prot_scaled_df = pd.concat([
    prot_sig_features[['PatientID', 'Timepoint']].reset_index(drop=True),
    pd.DataFrame(prot_scaled, columns=pnumeric_cols)
], axis=1)

meta_scaled_df = pd.concat([
    metab_sig_features[['PatientID', 'Timepoint']].reset_index(drop=True),
    pd.DataFrame(metab_scaled, columns=senumeric_cols)
], axis=1)


# --- 2. Check overlap of samples ---
# Step 1: Get common PatientIDs
bd_ids = set(bd_scaled_df['PatientID'])
prot_ids = set(prot_scaled_df['PatientID'])
meta_ids = set(meta_scaled_df['PatientID'])
#common_ids = bd_ids.intersection(prot_ids, meta_ids)  # if 2 overlapping samples: 11
common_ids = bd_ids.intersection(prot_ids, meta_ids)  # if 3 overlapping samples: 9

print("Number of overlapping samples:", len(common_ids))
print("Common sample IDs:", common_ids)

Number of overlapping samples: 9
Common sample IDs: {'RB01_007', 'RB01_012', 'RB01_014', 'RB01_005', 'RB01_009', 'RB01_010', 'RB01_013', 'RB01_008', 'RB01_006'}


In [107]:
# always run the prev block
# i need to change the ctrl vs stim column concat with all features
timepoint_map = {
    'Baseline': 0,
    'Day1': 1,
    'Day30': 2,
    'Day90': 3
}

# --- 1. Replace Timepoint with numeric in bd_scaled_df ---
bd_scaled_df['Timepoint'] = bd_scaled_df['Timepoint'].map(timepoint_map)
meta_scaled_df['Timepoint'] = meta_scaled_df['Timepoint'].map(timepoint_map)
# --- 2. Filter to common PatientIDs ---
#common_ids = set(bd_scaled_df['PatientID']).intersection(set(prot_scaled_df['PatientID']))
common_ids = (
    set(bd_scaled_df['PatientID'])
    .intersection(prot_scaled_df['PatientID'])
    .intersection(meta_scaled_df['PatientID'])
)

bd_common = bd_scaled_df[bd_scaled_df['PatientID'].isin(common_ids)].copy()
prot_common = prot_scaled_df[prot_scaled_df['PatientID'].isin(common_ids)].copy()
meta_common = meta_scaled_df[meta_scaled_df['PatientID'].isin(common_ids)].copy()

# --- 3. Sort both dataframes by PatientID and Timepoint to ensure alignment ---
bd_common = bd_common.sort_values(by=['PatientID','Timepoint']).reset_index(drop=True)
prot_common = prot_common.sort_values(by=['PatientID','Timepoint']).reset_index(drop=True)
meta_common = meta_common.sort_values(by=['PatientID','Timepoint']).reset_index(drop=True)
# --- 4. Drop metadata columns from prot except PatientID and Timepoint for merge ---
# (PatientID and Timepoint must exist for merge)
prot_numeric_only = prot_common.drop(columns=[])
meta_numeric_only = meta_common.drop(columns=[])

# --- 5. Merge on PatientID and Timepoint using inner join to keep only overlapping rows ---
df_integrated = pd.merge(
    bd_common,
    prot_common,
    on=['PatientID', 'Timepoint'],
    how='inner'
)

df_integrated = pd.merge(
    df_integrated,
    meta_numeric_only,
    on=['PatientID', 'Timepoint'],
    how='inner'
)


# --- 7. Check result ---
print("Final shape:", df_integrated.shape)
#print(df_integrated.head())
# save this in a file
#df_integrated.to_csv("run1/all2df_integrated.csv", index=True)
df_integrated.to_csv("run1/all3df_integrated.csv", index=True)

Final shape: (18, 264)


In [106]:
meta_scaled_df

,PatientID,Timepoint,cont_ATP production rate,cont_ATP restricted glycolisis,cont_ATP-linked respiration,cont_Basal glycolysis,cont_Basal mitochondrial respiration,cont_Maximal glycolitic capacity,cont_Maximal respiration,cont_OCR ATP,...,stim_Basal glycolysis,stim_Basal mitochondrial respiration,stim_Maximal glycolitic capacity,stim_Maximal respiration,stim_OCR ATP,stim_PER,stim_Phosphorylation-linked respiration,stim_glycoATP production rate (=glycoPER),stim_mitoOCR,stim_mitoPER
0,RB01_005,Baseline,-0.601914,-0.747645,0.232207,-0.747645,0.101043,-0.869632,1.662869,0.232207,...,-0.234048,-0.412776,0.159004,-0.172910,-0.220354,-0.234048,-0.202407,-0.339985,-0.412776,0.704764
1,RB01_005,Day1,2.258565,3.059592,0.854340,3.059592,0.653293,2.448865,1.043804,0.854340,...,2.259611,1.519742,2.262357,2.102787,1.806507,2.259611,1.704360,2.185261,1.519742,1.403542
2,RB01_005,Day30,-0.138889,-0.656613,-0.797411,-0.656613,-0.941135,-0.558049,-1.024488,-0.797411,...,-0.327921,-1.073428,-0.042212,-1.022850,-0.764507,-0.327921,-0.191732,-0.413344,-1.073428,0.516087
3,RB01_005,Day90,-0.269845,-0.077539,-0.152786,-0.077539,0.140532,-0.264543,0.994063,-0.152786,...,2.849748,2.396061,2.508893,1.034852,0.562906,2.849748,1.344681,2.986202,2.396061,0.047785
4,RB01_006,Baseline,-0.694152,-0.491227,-1.405299,-0.491227,-1.015935,-0.461217,-1.034961,-1.405299,...,-0.215722,-0.869640,-0.409724,-1.123656,-0.905391,-0.215722,-0.816863,-0.129667,-0.869640,-0.724677
5,RB01_006,Day1,-0.769581,-0.548366,-1.153832,-0.548366,-0.660239,-0.653986,-0.357534,-1.153832,...,0.014758,-0.272881,-0.142447,-0.248262,-0.289844,0.014758,-0.325991,0.103033,-0.272881,-0.654858
6,RB01_006,Day30,-1.110868,-0.582870,-0.388893,-0.582870,-0.924626,-0.438405,-0.338316,-0.388893,...,-0.465502,-1.062467,-0.418939,-0.893339,-1.011136,-0.465502,-0.791707,-0.397464,-1.062467,-0.683557
7,RB01_006,Day90,-0.536740,-0.780656,-0.387620,-0.780656,-0.823112,-0.971613,0.116151,-0.387620,...,-1.209754,-0.185323,-1.236430,0.219065,-0.052560,-1.209754,0.081339,-1.159137,-0.185323,-0.832308
8,RB01_007,Baseline,-0.263112,0.106701,0.749487,0.106701,0.644109,-0.027568,1.591751,0.749487,...,-0.439547,-1.350889,-0.413003,-1.180703,-1.122364,-0.439547,-1.032451,-0.265042,-1.350889,-1.470315
9,RB01_007,Day90,0.967966,0.352677,0.148920,0.352677,-0.004905,1.965698,0.909606,0.148920,...,0.551062,-1.207965,0.972515,-0.944294,-1.186502,0.551062,-1.020497,0.590052,-1.207965,-0.085043


In [96]:
df_integrated.shape[1]

252

In [97]:
#sedf

In [ ]:
# now can apply 
# MOFA+
# DIABLO (mixOmics)
# LME/ANOVA
